In [1]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
import tensorflow
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [4]:
def convert2matrix(data_arr, look_back):
   X, Y =[], []
   for i in range(len(data_arr)-look_back):
       d=i+look_back  
       X.append(data_arr[i:d,])
       Y.append(data_arr[d,])
   return np.array(X), np.array(Y)

In [5]:
df= pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STEST/S_1__I_582865.csv",parse_dates=['date'],index_col='date')

#convert date field from string to datetime
#df['date'] = pd.to_datetime(df['date'])

In [6]:
df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)

In [7]:
df=df[0:300]

In [8]:
k = int(len(df) * 0.8)
train,test = df.values[0:k,:], df.values[k:len(df.values),:]

In [9]:
look_back = 20 #create window size as look_back=30
test = np.append(test,np.repeat(test[-1,], look_back))
train = np.append(train,np.repeat(train[-1,],look_back))
trainX,trainY =convert2matrix(train,look_back)
testX,testY =convert2matrix(test, look_back)
# reshape input to be [samples, window size, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [11]:
testX.shape

(60, 1, 20)

In [13]:
def model_rnn(look_back):
  model=Sequential()
  model.add(SimpleRNN(units=32, input_shape=(1,look_back), activation="relu"))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
  return model

In [14]:
model=model_rnn(look_back)
history=model.fit(trainX,trainY, epochs=200, batch_size=30, verbose=1, validation_data=(testX,testY),callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)


Train on 240 samples, validate on 60 samples
Epoch 1/200
240/240 [==============================] - 1s 2ms/step - loss: 38.5370 - mse: 38.5370 - mae: 4.9188 - val_loss: 34.6855 - val_mse: 34.6855 - val_mae: 4.3750
Epoch 2/200
240/240 [==============================] - 0s 119us/step - loss: 34.1273 - mse: 34.1273 - mae: 4.5836 - val_loss: 29.6901 - val_mse: 29.6901 - val_mae: 3.9031
Epoch 3/200
240/240 [==============================] - 0s 126us/step - loss: 29.7464 - mse: 29.7465 - mae: 4.2623 - val_loss: 26.4654 - val_mse: 26.4654 - val_mae: 3.6488
Epoch 4/200
240/240 [==============================] - 0s 137us/step - loss: 27.5892 - mse: 27.5892 - mae: 4.0557 - val_loss: 24.1655 - val_mse: 24.1655 - val_mae: 3.4611
Epoch 5/200
240/240 [==============================] - 0s 196us/step - loss: 25.9768 - mse: 25.9768 - mae: 3.9564 - val_loss: 23.1975 - val_mse: 23.1975 - val_mae: 3.4284
Epoch 6/200
240/240 [==============================] - 0s 117us/step - loss: 24.9400 - mse: 24.9400 -

240/240 [==============================] - 0s 142us/step - loss: 16.7737 - mse: 16.7737 - mae: 3.2435 - val_loss: 14.5239 - val_mse: 14.5239 - val_mae: 2.6198
Epoch 48/200
240/240 [==============================] - 0s 134us/step - loss: 16.7102 - mse: 16.7102 - mae: 3.2402 - val_loss: 14.4651 - val_mse: 14.4651 - val_mae: 2.6119
Epoch 49/200
240/240 [==============================] - 0s 153us/step - loss: 16.6286 - mse: 16.6286 - mae: 3.2339 - val_loss: 14.4325 - val_mse: 14.4325 - val_mae: 2.6106
Epoch 50/200
240/240 [==============================] - 0s 197us/step - loss: 16.5563 - mse: 16.5563 - mae: 3.2219 - val_loss: 14.3975 - val_mse: 14.3975 - val_mae: 2.6079
Epoch 51/200
240/240 [==============================] - 0s 177us/step - loss: 16.5051 - mse: 16.5051 - mae: 3.2186 - val_loss: 14.3563 - val_mse: 14.3563 - val_mae: 2.5986
Epoch 52/200
240/240 [==============================] - 0s 146us/step - loss: 16.4143 - mse: 16.4143 - mae: 3.2134 - val_loss: 14.3841 - val_mse: 14.3841

In [16]:
train_predict = model.predict(trainX)
test_predict = model.predict(testX)

In [17]:
test_predict.shape

(60, 1)

In [15]:
test=np.asanyarray(Ytest)  
Ytest=Ytest.reshape(-1,1) 

NameError: name 'Ytest' is not defined

In [29]:
np.sqrt(mean_squared_error(testY,test_predict))

4.575339243322186

In [31]:
forecast1=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/ML_FORECAST_70%.csv",parse_dates=['date'])

In [32]:
forecast1

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,actual_value,LSTM_1,ANN,CNN
0,0,0,0,2013-08-20,15.910,12.966378,12.579720,16.562193
1,1,1,1,2013-08-21,10.261,12.850510,12.747951,14.999523
2,2,2,2,2013-08-22,15.035,13.017899,12.972117,16.905703
3,3,3,3,2013-08-23,12.190,12.846584,12.491053,13.831470
4,4,4,4,2013-08-24,11.514,12.917412,12.801822,7.564637
...,...,...,...,...,...,...,...,...
65,65,65,65,2013-10-24,13.996,12.674856,12.599476,6.641022
66,66,66,66,2013-10-25,15.341,12.811443,12.186004,12.548703
67,67,67,67,2013-10-26,9.206,12.779969,12.580894,15.209799
68,68,68,68,2013-10-27,12.771,13.015878,13.101615,14.770230


In [34]:
forecast1['RNN_2']=np.array(test_predict[20:])

In [33]:
len(test_predict)

90

In [35]:
forecast1

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,date,actual_value,LSTM_1,ANN,CNN,RNN_2
0,0,0,0,2013-08-20,15.910,12.966378,12.579720,16.562193,12.276929
1,1,1,1,2013-08-21,10.261,12.850510,12.747951,14.999523,13.190181
2,2,2,2,2013-08-22,15.035,13.017899,12.972117,16.905703,11.340872
3,3,3,3,2013-08-23,12.190,12.846584,12.491053,13.831470,14.544037
4,4,4,4,2013-08-24,11.514,12.917412,12.801822,7.564637,12.094693
...,...,...,...,...,...,...,...,...,...
65,65,65,65,2013-10-24,13.996,12.674856,12.599476,6.641022,12.206569
66,66,66,66,2013-10-25,15.341,12.811443,12.186004,12.548703,11.763103
67,67,67,67,2013-10-26,9.206,12.779969,12.580894,15.209799,11.140363
68,68,68,68,2013-10-27,12.771,13.015878,13.101615,14.770230,11.462310


In [36]:
forecast1.to_csv('ML_FORECAST_70%.csv')

In [26]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [28]:
dff=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/CNN+LSTM_forecast_error.csv")

In [34]:
import os
import logging, sys
logging.disable(sys.maxsize)
from math import sqrt
paths_to_folders = ['C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STORE__1']
x=0
for folder in paths_to_folders:
   for csv_file in os.listdir(folder):
        if  any(dff.File_name ==""+csv_file)==True:
       #if len(df.index)>=100:
            select_indices=list(np.where(dff["File_name"] == ''+csv_file)[0])
            print(select_indices)
            df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STORE__1/"+csv_file,parse_dates=['date'],index_col='date')
            df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)
            k = int(len(df) * 0.8)
            train,test = df.values[0:k,:], df.values[k:len(df.values),:]
            look_back = 10 #create window size as look_back=10
            test = np.append(test,np.repeat(test[-1,], look_back))
            train = np.append(train,np.repeat(train[-1,],look_back))
            trainX,trainY =convert2matrix(train,look_back)
            testX,testY =convert2matrix(test, look_back)
            # reshape input to be [samples, window size, features]
            trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
            testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
            model=model_rnn(look_back)
            history=model.fit(trainX,trainY, epochs=100, batch_size=30, verbose=1, validation_data=(testX,testY),callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
            test_predict = model.predict(testX)
            rmse = np.sqrt(mean_squared_error(testY,test_predict))
            error.loc[x, ['File_name']]=csv_file
            error.loc[x, ['RMSE_ERROR_RNN']]=rmse
            if len(select_indices)==1:
                b=dff.iloc[select_indices[0]]['RMSE_ERROR_LSTM']
                d=dff.iloc[select_indices[0]]['RMSE_ERROR_CNN']
                c=dff.iloc[select_indices[0]]['RMSE_ERROR_CNN+LSTM']
                error.loc[x, ['RMSE_ERROR_LSTM']]=b
                error.loc[x, ['RMSE_ERROR_CNN']]=d
                error.loc[x, ['RMSE_ERROR_CNN+LSTM']]=c
            error.loc[x, ['MSE']]=mean_squared_error(testY,test_predict)
            error.loc[x, ['MAPE']]=mean_absolute_percentage_error(testY,test_predict)
            x=x+1
            print(x)

[0]
Train on 101 samples, validate on 26 samples
Epoch 1/100
101/101 [==============================] - 1s 5ms/step - loss: 9.1832 - mse: 9.1832 - mae: 2.6297 - val_loss: 4.0485 - val_mse: 4.0485 - val_mae: 1.9021
Epoch 2/100
101/101 [==============================] - 0s 355us/step - loss: 8.5442 - mse: 8.5442 - mae: 2.5111 - val_loss: 3.6906 - val_mse: 3.6906 - val_mae: 1.8051
Epoch 3/100
101/101 [==============================] - 0s 278us/step - loss: 7.9181 - mse: 7.9181 - mae: 2.3891 - val_loss: 3.3440 - val_mse: 3.3440 - val_mae: 1.7053
Epoch 4/100
101/101 [==============================] - 0s 220us/step - loss: 7.3124 - mse: 7.3124 - mae: 2.2644 - val_loss: 3.0125 - val_mse: 3.0125 - val_mae: 1.6035
Epoch 5/100
101/101 [==============================] - 0s 410us/step - loss: 6.7360 - mse: 6.7360 - mae: 2.1386 - val_loss: 2.6907 - val_mse: 2.6907 - val_mae: 1.4981
Epoch 6/100
101/101 [==============================] - 0s 291us/step - loss: 6.1801 - mse: 6.1801 - mae: 2.0126 - val_

Epoch 15/100
80/80 [==============================] - 0s 271us/step - loss: 1.4373 - mse: 1.4373 - mae: 0.7999 - val_loss: 0.5254 - val_mse: 0.5254 - val_mae: 0.6347
Epoch 16/100
80/80 [==============================] - 0s 221us/step - loss: 1.3670 - mse: 1.3670 - mae: 0.7828 - val_loss: 0.4610 - val_mse: 0.4610 - val_mae: 0.5836
Epoch 17/100
80/80 [==============================] - 0s 171us/step - loss: 1.3095 - mse: 1.3095 - mae: 0.7695 - val_loss: 0.4085 - val_mse: 0.4085 - val_mae: 0.5363
Epoch 18/100
80/80 [==============================] - 0s 285us/step - loss: 1.2636 - mse: 1.2636 - mae: 0.7577 - val_loss: 0.3664 - val_mse: 0.3664 - val_mae: 0.4925
Epoch 19/100
80/80 [==============================] - 0s 314us/step - loss: 1.2290 - mse: 1.2290 - mae: 0.7479 - val_loss: 0.3335 - val_mse: 0.3335 - val_mae: 0.4570
Epoch 20/100
80/80 [==============================] - 0s 366us/step - loss: 1.2022 - mse: 1.2022 - mae: 0.7425 - val_loss: 0.3086 - val_mse: 0.3086 - val_mae: 0.4254
Epoc

97/97 [==============================] - 0s 229us/step - loss: 1.8022 - mse: 1.8022 - mae: 0.9360 - val_loss: 1.1612 - val_mse: 1.1612 - val_mae: 0.7425
Epoch 44/100
97/97 [==============================] - 0s 155us/step - loss: 1.7856 - mse: 1.7856 - mae: 0.9315 - val_loss: 1.1617 - val_mse: 1.1617 - val_mae: 0.7436
Epoch 45/100
97/97 [==============================] - 0s 286us/step - loss: 1.7694 - mse: 1.7694 - mae: 0.9281 - val_loss: 1.1602 - val_mse: 1.1602 - val_mae: 0.7433
Epoch 46/100
97/97 [==============================] - 0s 253us/step - loss: 1.7553 - mse: 1.7553 - mae: 0.9234 - val_loss: 1.1580 - val_mse: 1.1580 - val_mae: 0.7421
Epoch 47/100
97/97 [==============================] - 0s 353us/step - loss: 1.7415 - mse: 1.7415 - mae: 0.9186 - val_loss: 1.1549 - val_mse: 1.1549 - val_mae: 0.7414
Epoch 48/100
97/97 [==============================] - 0s 287us/step - loss: 1.7285 - mse: 1.7285 - mae: 0.9161 - val_loss: 1.1518 - val_mse: 1.1518 - val_mae: 0.7440
Epoch 49/100
97/9

5
[5]
Train on 80 samples, validate on 21 samples
Epoch 1/100
80/80 [==============================] - 1s 7ms/step - loss: 19.5984 - mse: 19.5984 - mae: 4.1237 - val_loss: 20.2351 - val_mse: 20.2351 - val_mae: 4.4275
Epoch 2/100
80/80 [==============================] - 0s 266us/step - loss: 17.0640 - mse: 17.0640 - mae: 3.8230 - val_loss: 17.7287 - val_mse: 17.7287 - val_mae: 4.1387
Epoch 3/100
80/80 [==============================] - 0s 401us/step - loss: 14.9965 - mse: 14.9965 - mae: 3.5554 - val_loss: 15.6491 - val_mse: 15.6491 - val_mae: 3.8818
Epoch 4/100
80/80 [==============================] - 0s 175us/step - loss: 13.2962 - mse: 13.2962 - mae: 3.3185 - val_loss: 13.9563 - val_mse: 13.9563 - val_mae: 3.6579
Epoch 5/100
80/80 [==============================] - 0s 307us/step - loss: 11.8731 - mse: 11.8731 - mae: 3.1067 - val_loss: 12.5558 - val_mse: 12.5558 - val_mae: 3.4624
Epoch 6/100
80/80 [==============================] - 0s 204us/step - loss: 10.7083 - mse: 10.7083 - mae: 2.

Epoch 50/100
80/80 [==============================] - 0s 164us/step - loss: 5.1176 - mse: 5.1176 - mae: 1.8538 - val_loss: 6.5483 - val_mse: 6.5483 - val_mae: 2.4528
Epoch 51/100
80/80 [==============================] - 0s 442us/step - loss: 5.1033 - mse: 5.1033 - mae: 1.8504 - val_loss: 6.5349 - val_mse: 6.5349 - val_mae: 2.4501
Epoch 52/100
80/80 [==============================] - 0s 148us/step - loss: 5.0890 - mse: 5.0890 - mae: 1.8471 - val_loss: 6.5216 - val_mse: 6.5216 - val_mae: 2.4474
Epoch 53/100
80/80 [==============================] - 0s 536us/step - loss: 5.0747 - mse: 5.0747 - mae: 1.8438 - val_loss: 6.5082 - val_mse: 6.5082 - val_mae: 2.4448
Epoch 54/100
80/80 [==============================] - 0s 276us/step - loss: 5.0606 - mse: 5.0606 - mae: 1.8405 - val_loss: 6.4950 - val_mse: 6.4950 - val_mae: 2.4421
Epoch 55/100
80/80 [==============================] - 0s 193us/step - loss: 5.0468 - mse: 5.0468 - mae: 1.8373 - val_loss: 6.4819 - val_mse: 6.4819 - val_mae: 2.4395
Epoc

...

Epoch 6/100
88/88 [==============================] - 0s 533us/step - loss: 2.2006 - mse: 2.2006 - mae: 1.1088 - val_loss: 1.0430 - val_mse: 1.0430 - val_mae: 0.6605
Epoch 7/100
88/88 [==============================] - 0s 639us/step - loss: 1.8722 - mse: 1.8722 - mae: 0.9797 - val_loss: 0.8752 - val_mse: 0.8752 - val_mae: 0.5386
Epoch 8/100
88/88 [==============================] - 0s 752us/step - loss: 1.5750 - mse: 1.5750 - mae: 0.8694 - val_loss: 0.7491 - val_mse: 0.7491 - val_mae: 0.4766
Epoch 9/100
88/88 [==============================] - 0s 415us/step - loss: 1.3283 - mse: 1.3283 - mae: 0.7751 - val_loss: 0.6758 - val_mse: 0.6758 - val_mae: 0.4904
Epoch 10/100
88/88 [==============================] - 0s 533us/step - loss: 1.1658 - mse: 1.1658 - mae: 0.7292 - val_loss: 0.6539 - val_mse: 0.6539 - val_mae: 0.5234
Epoch 11/100
88/88 [==============================] - 0s 1ms/step - loss: 1.0799 - mse: 1.0799 - mae: 0.7170 - val_loss: 0.6668 - val_mse: 0.6668 - val_mae: 0.5725
Epoch 12/1

Epoch 2/100
111/111 [==============================] - 0s 584us/step - loss: 5.3309 - mse: 5.3309 - mae: 1.8229 - val_loss: 3.6713 - val_mse: 3.6713 - val_mae: 1.4545
Epoch 3/100
111/111 [==============================] - 0s 593us/step - loss: 3.9612 - mse: 3.9612 - mae: 1.5111 - val_loss: 2.6349 - val_mse: 2.6349 - val_mae: 1.1251
Epoch 4/100
111/111 [==============================] - 0s 611us/step - loss: 3.1103 - mse: 3.1103 - mae: 1.3119 - val_loss: 2.0638 - val_mse: 2.0638 - val_mae: 0.9741
Epoch 5/100
111/111 [==============================] - ETA: 0s - loss: 2.0157 - mse: 2.0157 - mae: 0.992 - 0s 639us/step - loss: 2.6324 - mse: 2.6324 - mae: 1.1994 - val_loss: 1.7908 - val_mse: 1.7908 - val_mae: 0.9287
Epoch 6/100
111/111 [==============================] - 0s 530us/step - loss: 2.3916 - mse: 2.3916 - mae: 1.1491 - val_loss: 1.6832 - val_mse: 1.6832 - val_mae: 0.9312
Epoch 7/100
111/111 [==============================] - 0s 584us/step - loss: 2.2755 - mse: 2.2755 - mae: 1.1259 -

Epoch 52/100
104/104 [==============================] - 0s 570us/step - loss: 1.5599 - mse: 1.5599 - mae: 0.9130 - val_loss: 1.1423 - val_mse: 1.1423 - val_mae: 0.7001
Epoch 53/100
104/104 [==============================] - 0s 451us/step - loss: 1.5551 - mse: 1.5551 - mae: 0.9115 - val_loss: 1.1411 - val_mse: 1.1411 - val_mae: 0.6992
Epoch 54/100
104/104 [==============================] - 0s 1ms/step - loss: 1.5507 - mse: 1.5507 - mae: 0.9100 - val_loss: 1.1400 - val_mse: 1.1400 - val_mae: 0.6986
Epoch 55/100
104/104 [==============================] - 0s 451us/step - loss: 1.5458 - mse: 1.5458 - mae: 0.9085 - val_loss: 1.1392 - val_mse: 1.1392 - val_mae: 0.6982
Epoch 56/100
104/104 [==============================] - 0s 601us/step - loss: 1.5411 - mse: 1.5411 - mae: 0.9071 - val_loss: 1.1384 - val_mse: 1.1384 - val_mae: 0.6978
Epoch 57/100
104/104 [==============================] - 0s 673us/step - loss: 1.5364 - mse: 1.5364 - mae: 0.9057 - val_loss: 1.1371 - val_mse: 1.1371 - val_mae: 0

97
[97]
Train on 90 samples, validate on 23 samples
Epoch 1/100
90/90 [==============================] - 6s 68ms/step - loss: 4.5609 - mse: 4.5609 - mae: 1.7050 - val_loss: 1.6595 - val_mse: 1.6595 - val_mae: 0.9690
Epoch 2/100
90/90 [==============================] - 0s 743us/step - loss: 3.5466 - mse: 3.5466 - mae: 1.4472 - val_loss: 1.2993 - val_mse: 1.2993 - val_mae: 0.8420
Epoch 3/100
90/90 [==============================] - 0s 632us/step - loss: 2.7777 - mse: 2.7777 - mae: 1.2766 - val_loss: 1.0631 - val_mse: 1.0631 - val_mae: 0.7696
Epoch 4/100
90/90 [==============================] - 0s 554us/step - loss: 2.2522 - mse: 2.2522 - mae: 1.1776 - val_loss: 0.9358 - val_mse: 0.9358 - val_mae: 0.7137
Epoch 5/100
90/90 [==============================] - 0s 565us/step - loss: 1.9338 - mse: 1.9338 - mae: 1.1119 - val_loss: 0.8965 - val_mse: 0.8965 - val_mae: 0.6919
Epoch 6/100
90/90 [==============================] - 0s 532us/step - loss: 1.8039 - mse: 1.8039 - mae: 1.0765 - val_loss: 0.

Epoch 9/100
96/96 [==============================] - 0s 701us/step - loss: 1.6683 - mse: 1.6683 - mae: 0.9365 - val_loss: 0.6260 - val_mse: 0.6260 - val_mae: 0.6103
Epoch 10/100
96/96 [==============================] - 0s 524us/step - loss: 1.6176 - mse: 1.6176 - mae: 0.9418 - val_loss: 0.5970 - val_mse: 0.5970 - val_mae: 0.6127
Epoch 11/100
96/96 [==============================] - ETA: 0s - loss: 1.1881 - mse: 1.1881 - mae: 0.892 - 0s 723us/step - loss: 1.6075 - mse: 1.6075 - mae: 0.9581 - val_loss: 0.5904 - val_mse: 0.5904 - val_mae: 0.6227
Epoch 12/100
96/96 [==============================] - 0s 499us/step - loss: 1.6020 - mse: 1.6020 - mae: 0.9653 - val_loss: 0.5858 - val_mse: 0.5858 - val_mae: 0.6231
Epoch 13/100
96/96 [==============================] - 0s 649us/step - loss: 1.5844 - mse: 1.5844 - mae: 0.9609 - val_loss: 0.5764 - val_mse: 0.5764 - val_mae: 0.6152
Epoch 14/100
96/96 [==============================] - 0s 521us/step - loss: 1.5549 - mse: 1.5549 - mae: 0.9490 - val_lo

In [33]:
error = pd.DataFrame(columns=['File_name','RMSE_ERROR_CNN','RMSE_ERROR_LSTM','RMSE_ERROR_CNN+LSTM','RMSE_ERROR_RNN','MSE','MAPE'])

In [35]:
error.head()

,File_name,RMSE_ERROR_CNN,RMSE_ERROR_LSTM,RMSE_ERROR_CNN+LSTM,RMSE_ERROR_RNN,MSE,MAPE
0,S_1__I_1001305.csv,1.35458,1.10799,1.28191,0.819524,0.67162,39.0878
1,S_1__I_1009998.csv,1.47389,1.35859,0.874792,1.08118,1.16896,70.6793
2,S_1__I_1010752.csv,0.967724,1.00519,1.00439,0.479042,0.229481,19.399
3,S_1__I_1018617.csv,1.0624,1.35548,1.42294,1.08874,1.18536,48.4652
4,S_1__I_1040170.csv,1.32738,1.19962,0.95653,0.841786,0.708603,51.4315


In [36]:
error.to_csv('forecast_error_RNN.csv')